In [2]:
!pip install pyvi

     |████████████████████████████████| 8.5MB 5.4MB/s 
     |████████████████████████████████| 747kB 33.9MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import re
import math
from nltk.stem import PorterStemmer
import os
from pyvi import ViTokenizer, ViPosTagger
import pickle




class BuildIndex:

    b = 0.75
    k = 1.2

    def __init__(self, files):
        self.tf = {}
        self.df = {}
        self.filenames = files
        self.file_to_terms = self.process_files()
        self.regdex = self.regular_index(self.file_to_terms)
        self.invertedIndex = self.inverted_index()
        self.dltable = self.docLtable()
        self.dl = self.docLen()
        self.avgdl = self.avgdocl()
        self.N = self.doc_n()
        self.idf = self.inverse_df()

    def process_files(self):
        '''
        input: filenames
        output: a dictionary, with filename as key, and its term list as values 
        '''
        file_to_terms = {}

        for file in self.filenames:
            # read the whole text of a file into a single string with lowercase
            file_to_terms[file] = open(
                file, 'r', encoding="utf8").read().lower()
            # split text into words (tokenized list for a document)
            file_to_terms[file] = file_to_terms[file].split()

        return file_to_terms

    def doc_n(self):
        '''
        return the number of docs in the collection
        '''
        return len(self.file_to_terms)

    def avgdocl(self):
        sum = 0
        for file in self.dl.keys():
            sum += self.dl[file]
        avgdl = sum/len(self.dl.keys())
        return avgdl

    def index_one_file(self, termlist):
        '''
        map words to their position for one document
        input: termlist of a document.
        output: a dictionary with word as key, position as value.
        '''
        fileIndex = {}
        for index, word in enumerate(termlist):
            if word in fileIndex.keys():
                fileIndex[word].append(index)
            else:
                fileIndex[word] = [index]

        return fileIndex
   
    def regular_index(self, termlists):
        '''
        output: a dictionary. key: filename, 
                        value: a dictionary with word as key, position as value  
        '''
        regdex = {}

        for filename in termlists.keys():
            regdex[filename] = self.index_one_file(termlists[filename])

        return regdex

    def inverted_index(self):
        '''
        output: dictionary. key: word, 
                        value: a dictionary, key is filename, values is its term position of that file
        '''
        total_index = {}
        regdex = self.regdex

        for filename in regdex.keys():

            self.tf[filename] = {}

            for word in regdex[filename].keys():
                # tf dict key: filename, value: dict, key is word, value is count
                self.tf[filename][word] = len(regdex[filename][word])

                if word in self.df.keys():
                    # df dict key: word, value: counts of doc containing that word
                    self.df[word] += 1
                else:
                    self.df[word] = 1

                if word in total_index.keys():
                    if filename in total_index[word].keys():
                        total_index[word][filename].extend(
                            regdex[filename][word])
                    else:
                        total_index[word][filename] = regdex[filename][word]
                else:
                    total_index[word] = {filename: regdex[filename][word]}

        return total_index

    def docLtable(self):
        '''
        output: dict, key: word, value: dict(key: number of docs contaiing that word, value:total_freq)
        '''
        dltable = {}
        for w in self.invertedIndex.keys():
            total_freq = 0
            for file in self.invertedIndex[w].keys():
                total_freq += len(self.invertedIndex[w][file])

            dltable[w] = {len(self.invertedIndex[w].keys()): total_freq}

        return dltable

    def docLen(self):
        '''
        return a dict, key: filename, value: document length
        '''
        dl = {}
        for file in self.filenames:
            dl[file] = len(self.file_to_terms[file])
        return dl

    

    def inverse_df(self):
        '''
        output: inverse doc freq with key: word, value: idf
        '''
        idf = {}
        for w in self.df.keys():
            # idf[w] = math.log((self.N - self.df[w] + 0.5)/(self.df[w] + 0.5))
            idf[w] = math.log((self.N + 1)/self.df[w])
        return idf
#

    def get_score(self, filename, qlist):
        
        #filename: filename
        #qlist: term list of the query 
        #output: the score for a document
        
        score = 0
        for w in self.file_to_terms[filename]:
            if w not in qlist:
                continue
            wc = len(self.invertedIndex[w][filename])
            score += self.idf[w] * ((wc) * (self.k+1)) / (wc + self.k *
                                                          (1 - self.b + self.b * self.dl[filename] / self.avgdl))
        return score

    def BM25scores(self, qlist):
        
        #output: a dictionary with filename as key, score as value
        
        total_score = {}
        for doc in self.file_to_terms.keys():
            total_score[doc] = self.get_score(doc, qlist)
        return total_score

    def ranked_docs(self):
        ranked_docs = sorted(self.total_score.items(),
                             key=lambda x: x[1], reverse=True)
#

def saved_model(model):
    file2terms_pth = '/content/drive/MyDrive/CS521/Okapi BM25/file2terms.pkl'
    pickle.dump(model.file_to_terms, open(file2terms_pth, "wb"))
    
    InvertedIndex_pth = '/content/drive/MyDrive/CS521/Okapi BM25/InvertedIndex.pkl'
    pickle.dump(model.invertedIndex, open(InvertedIndex_pth, "wb"))

    dl_pth = '/content/drive/MyDrive/CS521/Okapi BM25/dl.pkl'
    pickle.dump(model.dl, open(dl_pth, "wb"))
    
    idf_pth = '/content/drive/MyDrive/CS521/Okapi BM25/idf.pkl'
    pickle.dump (model.idf, open(idf_pth, "wb"))

    avgdl_pth = '/content/drive/MyDrive/CS521/Okapi BM25/avgdl.pkl'
    pickle.dump(model.avgdl, open(avgdl_pth, "wb"))
    #dltable
    dltable_pth = '/content/drive/MyDrive/CS521/Okapi BM25/dltable.pkl'
    pickle.dump(model.dltable, open(dltable_pth, "wb"))

In [ ]:
lst_docs_paths = "/content/drive/MyDrive/CS521/DaXuLi1"
lst_docs = []
for r, s, f in os.walk(lst_docs_paths):
    for file in f:
        file_pth = lst_docs_paths + '/' + file
        lst_docs.append(file_pth)

a_file = open("/content/drive/MyDrive/CS521/Okapi BM25/files.pkl", "wb")
pickle. dump(lst_docs, a_file)
a_file. close()
#
print("Processing ...")
bm25_model = BuildIndex(lst_docs)
saved_model(bm25_model)

#
'''
idf = bm25_model.dl
a_file = open("./model_okapi_25/dl.pkl", "wb")
pickle. dump(idf, a_file)
a_file. close()


file2terms_pth = open('./model_okapi_25/file2terms.pkl', "wb")
pickle. dump(bm25_model.file_to_terms, file2terms_pth)
file2terms_pth. close()    

InvertedIndex_pth = open('./model_okapi_25/InvertedIndex.pkl', "wb")
pickle. dump(bm25_model.invertedIndex, InvertedIndex_pth)
InvertedIndex_pth. close()


dl_pth = './model_okapi_25/dl.pkl'
pickle.dump = (model.dl, open(dl_pth, "wb"))

avgdl_pth = './model_okapi_25/avgdl.pkl'
pickle.dump = (model.avgdl, open(avgdl_pth, "wb"))
#dltable
dltable_pth = './model_okapi_25/dltable.pkl'
pickle.dump = (model.dltable, open(dltable_pth, "wb"))
'''

Processing ...


'\nidf = bm25_model.dl\na_file = open("./model_okapi_25/dl.pkl", "wb")\npickle. dump(idf, a_file)\na_file. close()\n\n\nfile2terms_pth = open(\'./model_okapi_25/file2terms.pkl\', "wb")\npickle. dump(bm25_model.file_to_terms, file2terms_pth)\nfile2terms_pth. close()    \n\nInvertedIndex_pth = open(\'./model_okapi_25/InvertedIndex.pkl\', "wb")\npickle. dump(bm25_model.invertedIndex, InvertedIndex_pth)\nInvertedIndex_pth. close()\n\n\ndl_pth = \'./model_okapi_25/dl.pkl\'\npickle.dump = (model.dl, open(dl_pth, "wb"))\n\navgdl_pth = \'./model_okapi_25/avgdl.pkl\'\npickle.dump = (model.avgdl, open(avgdl_pth, "wb"))\n#dltable\ndltable_pth = \'./model_okapi_25/dltable.pkl\'\npickle.dump = (model.dltable, open(dltable_pth, "wb"))\n'

In [5]:
import re
import math
from nltk.stem import PorterStemmer
import os
from pyvi import ViTokenizer, ViPosTagger
import pickle

def load_weigt():
    with open('/content/drive/MyDrive/CS521/Okapi BM25/avgdl.pkl', 'rb') as f:
        avgdl = pickle.load(f)

    with open('/content/drive/MyDrive/CS521/Okapi BM25/dl.pkl', 'rb') as f:
        dl = pickle.load(f)
    
    with open('/content/drive/MyDrive/CS521/Okapi BM25/dltable.pkl', 'rb') as f:
        dltable = pickle.load(f)
    
    with open('/content/drive/MyDrive/CS521/Okapi BM25/file2terms.pkl', 'rb') as f:
        f2t = pickle.load(f)
    
    with open('/content/drive/MyDrive/CS521/Okapi BM25/files.pkl', 'rb') as f:
        files = pickle.load(f)

    with open('/content/drive/MyDrive/CS521/Okapi BM25/idf.pkl', 'rb') as f:
        idf = pickle.load(f)
    
    with open('/content/drive/MyDrive/CS521/Okapi BM25/InvertedIndex.pkl', 'rb') as f:
        iDex = pickle.load(f)

    return files, f2t, iDex, dltable, dl, avgdl, idf


def preprocessing_query(query):
    dict_path =r'/content/drive/MyDrive/CS521/pro_dictionary (1).txt'
    with open(dict_path, encoding='utf8') as fobj:
        stop_words=fobj.read()
    query = ViTokenizer.tokenize(query)
    query = query.lower()
    # split text into words (tokenized list of a document)
    
    query = query.split()
    stop_words = stop_words.split()
    prew = ''
    for word in query:
        if word not in stop_words:
            prew = prew + word + ' '
    query = prew.split()

    return query


def getsc(filename, file_to_terms,idf, invertedIndex, k, b, dl, avgdl, qlist):
    score = 0
    for w in file_to_terms[filename]:
        if w not in qlist:
            continue
        wc = len(invertedIndex[w][filename])
        score += idf[w] * ((wc) * (k+1)) / (wc + k *
                                                        (1 - b + b * dl[filename] / avgdl))
    return score

def BM25scores(qlist, file_to_terms, idf, invertedIndex, k, b, dl, avgdl):
        '''
        output: a dictionary with filename as key, score as value
        '''
        total_score = {}
        for doc in file_to_terms.keys():

            total_score[doc] = getsc(doc, file_to_terms, idf, invertedIndex, k, b, dl, avgdl, qlist)
        
        return total_score

def ranked_docs(total_score):
    ranked_docs = sorted(total_score.items(),
                            key=lambda x: x[1], reverse=True)
    return ranked_docs

query = "điều 211"

files, f2t, iDex, dltable, dl, avgdl, idf = load_weigt()
query = preprocessing_query(query)
k = 1.2
b = 0.75
sc = BM25scores(query, f2t, idf, iDex, k  , b , dl, avgdl)
print(ranked_docs(sc))

[('/content/drive/MyDrive/CS521/DaXuLi1/Dieu1.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu3.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu2.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu4.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu11.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu5.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu6.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu7.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu8.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu10.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu9.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu12.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu13.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu14.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu15.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu16.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu17.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu18.txt', 0), 

In [6]:
query = "điều 211"
q1=query.split(" ")
query=''
for i in q1:
  if i=="mục" or i=="điều" or i=="chương":
    query=query+i+"_"
  else:
    query=query+i+" "
query=query[:-1]
files, f2t, iDex, dltable, dl, avgdl, idf = load_weigt()
query = preprocessing_query(query)
k = 1.2
b = 0.75
sc = BM25scores(query, f2t, idf, iDex, k  , b , dl, avgdl)
print(ranked_docs(sc))
#print(sc.keys())


[('/content/drive/MyDrive/CS521/DaXuLi1/Dieu211.txt', 6.068772577947281), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu1.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu3.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu2.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu4.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu11.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu5.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu6.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu7.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu8.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu10.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu9.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu12.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu13.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu14.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu15.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu16.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/

In [7]:
a = [('/content/drive/MyDrive/CS521/DaXuLi1/Dieu211.txt', 6.068772577947281), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu1.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu3.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu2.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu4.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu11.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu5.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu6.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu7.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu8.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu10.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu9.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu12.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu13.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu14.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu15.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu16.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu17.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu18.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu19.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu20.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu21.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu22.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu23.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu24.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu25.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu26.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu27.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu28.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu29.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu30.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu31.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu32.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu33.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu34.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu35.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu36.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu37.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu38.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu39.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu40.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu41.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu42.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu43.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu44.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu45.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu46.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu47.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu48.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu49.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu50.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu51.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu52.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu53.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu54.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu55.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu56.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu57.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu58.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu59.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu60.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu77.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu76.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu75.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu74.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu73.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu72.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu71.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu70.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu69.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu68.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu67.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu66.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu65.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu64.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu63.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu62.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu61.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu78.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu79.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu80.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu81.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu82.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu83.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu84.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu85.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu86.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu87.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu88.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu89.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu90.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu91.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu92.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu93.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu94.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu95.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu96.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu97.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu98.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu99.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu100.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu101.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu102.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu103.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu104.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu105.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu106.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu107.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu108.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu109.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu110.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu111.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu112.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu113.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu114.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu115.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu116.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu117.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu118.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu119.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu120.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu121.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu122.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu123.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu124.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu128.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu127.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu126.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu125.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu129.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu130.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu131.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu132.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu133.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu134.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu135.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu136.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu137.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu138.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu139.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu140.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu141.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu142.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu143.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu144.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu145.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu146.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu147.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu148.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu149.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu150.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu151.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu152.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu153.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu154.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu155.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu158.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu159.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu160.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu161.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu162.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu163.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu164.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu165.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu166.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu167.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu168.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu169.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu170.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu171.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu172.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu173.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu174.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu175.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu176.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu177.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu178.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu179.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu180.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu157.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu156.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu181.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu182.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu183.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu184.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu185.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu186.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu187.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu188.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu189.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu190.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu191.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu192.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu193.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu194.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu195.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu196.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu197.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu198.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu199.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu200.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu201.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu202.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu203.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu204.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu205.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu206.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu207.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu208.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu209.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu210.txt', 0), ('/content/drive/MyDrive/CS521/DaXuLi1/Dieu212.txt', 0)]


In [11]:
a=ranked_docs(sc)

In [14]:
#a=ranked_docs(sc)
for i in a:
  path=i[0]
  new_path= path.split('/')[-1]
  path='/content/drive/MyDrive/CS521/LuatDatDai2013/'+new_path
  #print(path)
  with open(path, 'r', encoding="utf8") as f:
      full_doc = f.read()
      print(full_doc[:300])
      print("...................................................")

Chương 14.
ĐIỀU KHOẢN THI HÀNH
Điều 211. Hiệu lực thi hành
1. Luật này có hiệu lực thi hành từ ngày 01 tháng 7 năm 2014.
Luật đất đai số 13/2003/QH11 và Nghị quyết số 49/2013/QH13 ngày 21 tháng 6 năm 2013 của Quốc hội về kéo dài thời hạn sử dụng đất trồng cây hàng năm, đất nuôi trồng thủy sản, đất l
...................................................
Chương 1.
QUY ĐỊNH CHUNG
Điều 1. Phạm vi điều chỉnh
Luật này quy định về chế độ sở hữu đất đai, quyền hạn và trách nhiệm của Nhà nước đại diện chủ sở hữu toàn dân về đất đai và thống nhất quản lý về đất đai, chế độ quản lý và sử dụng đất đai, quyền và nghĩa vụ của người sử dụng đất đối với đất đai t
...................................................
Chương 1.
QUY ĐỊNH CHUNG
Điều 3. Giải thích từ ngữ
Trong Luật này, các từ ngữ dưới đây được hiểu như sau:
1. Thửa đất là phần diện tích đất được giới hạn bởi ranh giới xác định trên thực địa hoặc được mô tả trên hồ sơ.
2. Quy hoạch sử dụng đất là việc phân bổ và khoanh vùng đất đai theo không 